In [1]:
from encoder_paths import *
import json
from pprint import pprint
TRAIN_FILE_PATH = "/tmp/semeval24_task3/SemEval-2024_Task3/official_data/Training_data/text/training.json"
VALIDATION_FILE_PATH = "/tmp/semeval24_task3/SemEval-2024_Task3/official_data/Training_data/text/testing.json"
with open(TRAIN_FILE_PATH) as f:
    train_data = json.load(f)
with open(VALIDATION_FILE_PATH) as f:
    validation_data = json.load(f)

pprint(len(train_data))
pprint(len(validation_data))

1236
138


In [2]:
import numpy
import random
import torch

numpy.random.seed(69)
random.seed(69)
torch.manual_seed(69)

/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
all_emotions = set()
for conversation in train_data:
    conversation = conversation["conversation"]
    for utterance in conversation:
        all_emotions.add(utterance["emotion"])
pprint(all_emotions)

{'surprise', 'joy', 'fear', 'sadness', 'disgust', 'anger', 'neutral'}


In [4]:
class EmotionIndexer:
    def __init__(self):
        self.emotion_to_index = {
            'joy': 0,
            'sadness': 1,
            'anger': 2,
            'neutral': 3,
            'surprise': 4,
            'disgust': 5,
            'fear': 6,
        }

        self.index_to_emotion = {index: emotion for emotion, index in self.emotion_to_index.items()}

    def emotion_to_idx(self, emotion):
        return self.emotion_to_index.get(emotion, None)

    def idx_to_emotion(self, index):
        return self.index_to_emotion.get(index, None)

# Example usage
indexer = EmotionIndexer()


In [5]:
import torch
import json
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_video
from torchvision.transforms import functional as F
from PIL import Image
import numpy as np

In [6]:
import torch
import pickle

class YourAudioEncoder():
    def __init__(self, audio_embeddings_path):
        with open(audio_embeddings_path, "rb") as f:
            self.audio_embeddings = pickle.load(f)

    def lmao(self, audio_name):
        audio_name = audio_name.split(".")[0]
        audio_embedding = self.audio_embeddings[audio_name]
        audio_embedding = audio_embedding.squeeze()
        return torch.from_numpy(audio_embedding)
    
class YourVideoEncoder():
    def __init__(self, video_embeddings_path):
        with open(video_embeddings_path, "rb") as f:
            self.video_embeddings = pickle.load(f)

    def lmao(self, video_name):
        # video_name = video_name.split(".")[0]
        video_embedding = self.video_embeddings[video_name].reshape((16,-1))
        video_embedding = np.mean(video_embedding, axis=0)
        return torch.from_numpy(video_embedding)

class YourTextEncoder():
    def __init__(self, text_embeddings_path):
        with open(text_embeddings_path, "rb") as f:
            self.text_embeddings = pickle.load(f)

    def lmao(self, video_name):
        text_embedding = self.text_embeddings[video_name]
        return torch.from_numpy(text_embedding)


In [7]:
class ConversationDataset(Dataset):
    def __init__(self, json_file, audio_encoder, video_encoder, text_encoder, max_seq_len):
        self.max_seq_len = max_seq_len
        self.data = self.load_data(json_file)
        self.audio_encoder = audio_encoder
        self.video_encoder = video_encoder
        self.text_encoder = text_encoder

    def load_data(self, json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        conversation = self.data[idx]['conversation']
        # emotion_labels = [utterance['emotion'] for utterance in conversation]
        audio_paths = [utterance['video_name'].replace('mp4', 'wav') for utterance in conversation]
        video_paths = [utterance['video_name'] for utterance in conversation]
        texts = [utterance['video_name'] for utterance in conversation]

        audio_embeddings = [self.audio_encoder.lmao(audio_path) for audio_path in audio_paths]
        video_embeddings = [self.video_encoder.lmao(video_path) for video_path in video_paths]
        text_embeddings = [self.text_encoder.lmao(text) for text in texts]

        cause_pairs = self.data[idx]['emotion-cause_pairs']
        useful_utterances = set([int(cause_pair[1]) for cause_pair in cause_pairs])
        cause_labels = []
        for utterance in conversation:
            if utterance['utterance_ID'] in useful_utterances:
                cause_labels.append(1)
            else:
                cause_labels.append(0)
        
        # Pad or truncate conversations to the maximum sequence length
        if len(conversation) < self.max_seq_len:
            pad_length = self.max_seq_len - len(conversation)
            audio_embeddings += [torch.zeros_like(audio_embeddings[0])] * pad_length
            video_embeddings += [torch.zeros_like(video_embeddings[0])] * pad_length
            text_embeddings += [torch.zeros_like(text_embeddings[0])] * pad_length
            cause_labels += [-1] * pad_length
            pad_mask = [1] * len(conversation) + [0] * pad_length
        else:
            audio_embeddings = audio_embeddings[:self.max_seq_len]
            video_embeddings = video_embeddings[:self.max_seq_len]
            text_embeddings = text_embeddings[:self.max_seq_len]
            cause_labels = cause_labels[:self.max_seq_len]
            pad_mask = [1] * self.max_seq_len

        audio_embeddings = torch.stack(audio_embeddings)
        video_embeddings = torch.stack(video_embeddings)
        text_embeddings = torch.stack(text_embeddings)
        cause_labels = torch.from_numpy(np.array(cause_labels))
        pad_mask = torch.from_numpy(np.array(pad_mask))
        
        return {
            'audio': audio_embeddings,
            'video': video_embeddings,
            'text': text_embeddings,
            'cause_labels': cause_labels,
            'pad_mask': pad_mask
        }

# Example usage
# You need to define your audio, video, and text encoders accordingly

# Define your data paths
# DATA_DIR = "/tmp/semeval24_task3"

# AUDIO_EMBEDDINGS_FILEPATH = "/tmp/semeval24_task3/og_paper_embeddings/audio_embedding_6373.npy"
# VIDEO_EMBEDDINGS_FILEPATH = "/tmp/semeval24_task3/og_paper_embeddings/video_embedding_4096.npy"
# TEXT_EMBEDDINGS_FILEPATH = os.path.join(DATA_DIR, "text_embeddings", "text_embeddings_bert_base.pkl")

audio_encoder = YourAudioEncoder(AUDIO_EMBEDDINGS_FILEPATH)
video_encoder = YourVideoEncoder(VIDEO_EMBEDDINGS_FILEPATH)
text_encoder = YourTextEncoder(TEXT_EMBEDDINGS_FILEPATH)
max_seq_len = 35  # Adjust this according to your needs

train_dataset = ConversationDataset(TRAIN_FILE_PATH, audio_encoder, video_encoder, text_encoder, max_seq_len)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

validation_dataset = ConversationDataset(VALIDATION_FILE_PATH, audio_encoder, video_encoder, text_encoder, max_seq_len)
validation_dataloader = DataLoader(validation_dataset, batch_size=16, shuffle=True)
# Example of iterating through batches
# for batch in dataloader:
#     audio = batch['audio']  # Shape: (batch_size, max_seq_len, audio_embedding_size)
#     video = batch['video']  # Shape: (batch_size, max_seq_len, video_embedding_size)
#     text = batch['text']    # Shape: (batch_size, max_seq_len, text_embedding_size)
#     cause_labels = batch['cause_labels']  # List of emotion labels for each utterance in the batch


In [8]:
# import torch
# import torch.nn as nn
# from torch.nn import TransformerEncoder, TransformerEncoderLayer

# class EmotionClassifier(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_heads, dropout, num_emotions):
#         super(EmotionClassifier, self).__init__()
        
#         self.first_linear = nn.Linear(input_size, hidden_size, dtype=torch.float32)

#         self.transformer_encoder = TransformerEncoder(
#             TransformerEncoderLayer(hidden_size, num_heads, hidden_size, dropout),
#             num_layers
#         )
        
#         self.linear = nn.Linear(hidden_size, num_emotions)

#     def forward(self, audio_encoding, video_encoding, text_encoding):

#         # Concatenate or combine the audio, video, and text encodings here
#         # You can use any method like concatenation, addition, or other fusion techniques
#         # Combine the encodings (you can customize this part)
#         audio_encoding = audio_encoding.float()
#         video_encoding = video_encoding.float()
#         text_encoding = text_encoding.float().squeeze()
#         combined_encoding = torch.cat((audio_encoding, video_encoding, text_encoding), dim=2)
        
#         combined_encoding = self.first_linear(combined_encoding)
        
        
#         combined_encoding = combined_encoding.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, input_size)
        
        
#         transformer_output = self.transformer_encoder(combined_encoding)

#         # Take the output of the Transformer encoder for the last position as the summary
#         emotion_logits = self.linear(transformer_output.permute(1, 0, 2))

#         return emotion_logits

In [9]:
import torch
import torch.nn as nn

class EmotionClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, num_emotions, embedding_dropout=0.2):
        super(EmotionClassifier, self).__init__()
        
        self.audio_dropout = nn.Dropout(embedding_dropout)
        self.video_dropout = nn.Dropout(embedding_dropout)
        self.text_dropout = nn.Dropout(embedding_dropout)

        # self.first_linear = nn.Linear(input_size, hidden_size, dtype=torch.float32)
        self.relu = nn.ReLU()
        
        # self.second_linear_layer = nn.Linear(hidden_size, hidden_size, dtype=torch.float32)
        # Replace Transformer with BiLSTM
        self.bilstm = nn.LSTM(input_size, input_size // 2, num_layers, 
                              dropout=dropout, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(input_size, hidden_size)
        self.final_linear = nn.Linear(hidden_size, num_emotions)

    def forward(self, audio_encoding, video_encoding, text_encoding):
        # Concatenate or combine the audio, video, and text encodings
        audio_encoding = audio_encoding.float()
        video_encoding = video_encoding.float()
        text_encoding = text_encoding.float().squeeze()
        
        audio_encoding = self.audio_dropout(audio_encoding)
        video_encoding = self.video_dropout(video_encoding)
        text_encoding = self.text_dropout(text_encoding)
        
        combined_encoding = torch.cat((audio_encoding, video_encoding, text_encoding), dim=2)
        
        # Pass through BiLSTM
        lstm_output, _ = self.bilstm(combined_encoding)

        # Take the output of the BiLSTM
        emotion_logits = self.linear(lstm_output)
        emotion_logits = self.relu(emotion_logits)
        emotion_logits = self.final_linear(emotion_logits)
        # Apply a softmax layer
        emotion_logits = torch.softmax(emotion_logits, dim=2)

        return emotion_logits

In [10]:
# import torch
# import torch.nn as nn

# class EmotionClassifier(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, dropout, num_emotions, embedding_dropout=0.2):
#         super(EmotionClassifier, self).__init__()
        
#         self.audio_dropout = nn.Dropout(embedding_dropout)
#         self.video_dropout = nn.Dropout(embedding_dropout)
#         self.text_dropout = nn.Dropout(embedding_dropout)

#         self.first_linear = nn.Linear(input_size, hidden_size, dtype=torch.float32)
#         self.relu = nn.ReLU()
        
#         self.second_linear_layer = nn.Linear(hidden_size, hidden_size, dtype=torch.float32)
#         # Replace Transformer with BiLSTM
#         self.bilstm = nn.LSTM(hidden_size, hidden_size // 2, num_layers, 
#                               dropout=dropout, bidirectional=True, batch_first=True)
        
#         self.linear = nn.Linear(hidden_size, num_emotions)

#     def forward(self, audio_encoding, video_encoding, text_encoding):
#         # Concatenate or combine the audio, video, and text encodings
#         audio_encoding = audio_encoding.float()
#         video_encoding = video_encoding.float()
#         text_encoding = text_encoding.float().squeeze()
        
#         audio_encoding = self.audio_dropout(audio_encoding)
#         video_encoding = self.video_dropout(video_encoding)
#         text_encoding = self.text_dropout(text_encoding)
        
#         combined_encoding = torch.cat((audio_encoding, video_encoding, text_encoding), dim=2)
        
#         combined_encoding = self.first_linear(combined_encoding)
#         combined_encoding = self.relu(combined_encoding)
#         combined_encoding = self.second_linear_layer(combined_encoding)
        
#         # Pass through BiLSTM
#         lstm_output, _ = self.bilstm(combined_encoding)

#         # Take the output of the BiLSTM
#         emotion_logits = self.linear(lstm_output)
#         # Apply a softmax layer
#         emotion_logits = torch.softmax(emotion_logits, dim=2)

#         return emotion_logits

In [11]:
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import classification_report
from transformers import get_linear_schedule_with_warmup

# Define your model
model = EmotionClassifier(input_size=768+1024*2, hidden_size=2000, num_emotions=2, num_layers=2, dropout=0.3)
model.to("cuda:1")

num_epochs = 40
# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = AdamW(model.parameters(), lr=0.5*1e-5)
total_steps = len(train_dataloader) * num_epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Define training parameters

# Training loop
best_model_file = None
best_val_loss = float('inf')
best_epoch = -1
best_classification_report = None

print(AUDIO_EMBEDDINGS_FILEPATH)
print(TEXT_EMBEDDINGS_FILEPATH)
print(VIDEO_EMBEDDINGS_FILEPATH)
for epoch in (range(num_epochs)):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    total_tokens = 0
    total_correct = 0
    total_predictions = 0

    for batch in tqdm(train_dataloader):  # Assuming you have a DataLoader for your dataset
        # Extract data from the batch
        audio = batch['audio'].to('cuda:1')
        video = batch['video'].to('cuda:1')
        text = batch['text'].to('cuda:1')
        emotion_indices = batch['cause_labels'].to('cuda:1')
        pad_mask = batch['pad_mask'].to('cuda:1')

        # Forward pass
        emotion_logits = model(audio, video, text)

        # Reshape emotion_logits
        emotion_logits = emotion_logits.view(-1, emotion_logits.size(-1))

        # Flatten emotion_indices (assuming it's a 2D tensor with shape [batch_size, max_sequence_length])
        emotion_indices = emotion_indices.view(-1)

        # Calculate a mask to exclude padded positions from the loss
        pad_mask = pad_mask.view(-1)     

        # Calculate the loss, excluding padded positions
        loss = criterion(emotion_logits, emotion_indices)
        # masked_loss = torch.sum(loss * pad_mask) / torch.sum(pad_mask)
        masked_loss = loss

        # Backpropagation and optimization
        optimizer.zero_grad()
        masked_loss.backward()
        optimizer.step()

        total_loss += masked_loss.item()
        total_tokens += torch.sum(pad_mask).item()
        
        predicted_emotions = torch.argmax(emotion_logits, dim=1)
        correct_predictions = ((predicted_emotions == emotion_indices) * pad_mask).sum().item()

        total_correct += correct_predictions
        total_predictions += torch.sum(pad_mask).item()  # Batch size

    scheduler.step()
    
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0.0
    total_val_tokens = 0
    total_val_correct = 0
    total_val_predictions = 0
    true_labels = []
    predicted_labels = []
    padded_labels = []
    
    with torch.no_grad():
        for val_batch in tqdm(validation_dataloader):
            audio = val_batch['audio'].to('cuda:1')
            video = val_batch['video'].to('cuda:1')
            text = val_batch['text'].to('cuda:1')
            emotion_indices = val_batch['cause_labels'].to('cuda:1')
            pad_mask = val_batch['pad_mask'].to('cuda:1')

            emotion_logits = model(audio, video, text)

            # Reshape emotion_logits
            emotion_logits = emotion_logits.view(-1, emotion_logits.size(-1))

            # Flatten emotion_indices (assuming it's a 2D tensor with shape [batch_size, max_sequence_length])
            emotion_indices = emotion_indices.view(-1)

            pad_mask = pad_mask.view(-1)   

            # Calculate the loss, excluding padded positions
            val_loss = criterion(emotion_logits, emotion_indices)
            masked_loss = torch.sum(val_loss * pad_mask) / torch.sum(pad_mask)
            
            total_val_loss += masked_loss.item()
            total_val_tokens += torch.sum(pad_mask).item()
            
            predicted_emotions_val = torch.argmax(emotion_logits, dim=1)
            correct_predictions_val = ((predicted_emotions_val == emotion_indices) * pad_mask).sum().item()
            total_val_correct += correct_predictions_val
            total_val_predictions += torch.sum(pad_mask).item()

            # Store true and predicted labels for F1 score calculation
            true_labels.extend(emotion_indices.cpu().numpy())
            predicted_labels.extend(predicted_emotions_val.cpu().numpy())
            padded_labels.extend(pad_mask.cpu().numpy())

    final_true_labels = [label for label, pad in zip(true_labels, padded_labels) if pad == 1]
    final_predicted_labels = [label for label, pad in zip(predicted_labels, padded_labels) if pad == 1]
    classification_rep = classification_report(final_true_labels, final_predicted_labels)

    # Calculate and print the average loss for this epoch
    avg_loss = total_loss / total_tokens
    avg_val_loss = total_val_loss / total_val_tokens
    print(f"Epoch [{epoch}/{num_epochs}] Training Loss: {avg_loss}")
    print(f"Epoch [{epoch}/{num_epochs}] Validation Loss: {avg_val_loss}")
    print(f"Epoch [{epoch}/{num_epochs}] Classification Report:\n{classification_rep}")
    print(f"Epoch [{epoch}/{num_epochs}] Accuracy: {total_correct / total_predictions:.4f}")
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch = epoch
        best_classification_report = classification_rep
        best_model_file = f"/tmp/semeval24_task3/baseline_models/cause_models/best_cause_model.pt"
        torch.save(model.state_dict(), best_model_file)

    torch.save(model.state_dict(), f"/tmp/semeval24_task3/baseline_models/cause_models/cause_model_{epoch:02}.pt")

print("Training complete!")
print("=======================================")
print("BEST MODEL")
print(f"Best epoch: {best_epoch}")
print(f"Best validation loss: {best_val_loss}")
print(f"Best classification report:\n{best_classification_report}")
print("=======================================")

/tmp/semeval24_task3/audio_embeddings/audio_embeddings_facebook_wav2vec2-large-960h.pkl
/tmp/semeval24_task3/text_embeddings/text_embeddings_roberta_large.pkl
/tmp/semeval24_task3/video_embeddings/final_embeddings.pkl


100%|██████████| 9/9 [00:00<00:00, 27.19it/s]
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter 

Epoch [0/40] Training Loss: 0.004432162052011713
Epoch [0/40] Validation Loss: 0.004313827181855838
Epoch [0/40] Classification Report:
              precision    recall  f1-score   support

           0       0.53      1.00      0.69       759
           1       0.00      0.00      0.00       681

    accuracy                           0.53      1440
   macro avg       0.26      0.50      0.35      1440
weighted avg       0.28      0.53      0.36      1440

Epoch [0/40] Accuracy: 0.5178


100%|██████████| 9/9 [00:00<00:00, 27.94it/s]


Epoch [1/40] Training Loss: 0.004421530167098625
Epoch [1/40] Validation Loss: 0.004305437869495816
Epoch [1/40] Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.96      0.69       759
           1       0.66      0.08      0.15       681

    accuracy                           0.55      1440
   macro avg       0.60      0.52      0.42      1440
weighted avg       0.60      0.55      0.43      1440

Epoch [1/40] Accuracy: 0.5311


...

Epoch [37/40] Training Loss: 0.00344978559828615
Epoch [37/40] Validation Loss: 0.0039352566003799435
Epoch [37/40] Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.62      0.66       759
           1       0.62      0.70      0.66       681

    accuracy                           0.66      1440
   macro avg       0.66      0.66      0.66      1440
weighted avg       0.66      0.66      0.66      1440

Epoch [37/40] Accuracy: 0.7650


100%|██████████| 9/9 [00:00<00:00, 30.11it/s]


Epoch [38/40] Training Loss: 0.0034564683414639017
Epoch [38/40] Validation Loss: 0.003917074493236012
Epoch [38/40] Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.64      0.66       759
           1       0.62      0.68      0.65       681

    accuracy                           0.65      1440
   macro avg       0.65      0.66      0.65      1440
weighted avg       0.66      0.65      0.65      1440

Epoch [38/40] Accuracy: 0.7628


100%|██████████| 9/9 [00:00<00:00, 30.44it/s]


Epoch [39/40] Training Loss: 0.0034487429525731027
Epoch [39/40] Validation Loss: 0.0038887867910994424
Epoch [39/40] Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.64      0.66       759
           1       0.63      0.67      0.65       681

    accuracy                           0.65      1440
   macro avg       0.65      0.65      0.65      1440
weighted avg       0.66      0.65      0.65      1440

Epoch [39/40] Accuracy: 0.7659
Training complete!
BEST MODEL
Best epoch: 21
Best validation loss: 0.0038884758949279787
Best classification report:
              precision    recall  f1-score   support

           0       0.66      0.70      0.68       759
           1       0.65      0.60      0.62       681

    accuracy                           0.65      1440
   macro avg       0.65      0.65      0.65      1440
weighted avg       0.65      0.65      0.65      1440

